In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def mes_a_numero(nombre_mes):
    # Diccionario que mapea el nombre del mes al número correspondiente
    meses = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }

    # Convertir el nombre del mes a minúsculas y buscar el número correspondiente
    return meses.get(nombre_mes.lower(), 'Mes inválido')

In [6]:
fecha_ida_usuario = '02/02/2025'
fecha_vuelta_usuario = '03/02/2025'
origen_usuario = 'Madrid'
destino_usuario = 'Barcelona'

driver = webdriver.Chrome()
esperar = WebDriverWait(driver, 10)
driver.get('https://www.alsa.es')
driver.implicitly_wait(1)
cookies = driver.find_element(By.CSS_SELECTOR, '#didomi-notice-agree-button')
cookies.click()

#origen
origen = driver.find_element(By.ID, '_originStationNameId_')
origen.click()
buscador = driver.find_element(By.CSS_SELECTOR, '#_JourneySearchPortlet_WAR_Alsaportlet_INSTANCE_JourneySearch_21651890_originStationNameId')
buscador.click()
buscador.send_keys(origen_usuario)
ul_element = esperar.until(EC.visibility_of_element_located((By.ID, "ui-id-1")))
primer_elemento = ul_element.find_element(By.CSS_SELECTOR, "li:first-child")
primer_elemento.click()    

#destino
destino = driver.find_element(By.ID, '_destinationStationNameId_')
destino.click()
buscador_destino = driver.find_element(By.CSS_SELECTOR, '#_JourneySearchPortlet_WAR_Alsaportlet_INSTANCE_JourneySearch_21651890_destinationStationNameId')
buscador_destino.click()
buscador_destino.send_keys(destino_usuario)
destino_ul_element = esperar.until(EC.visibility_of_element_located((By.ID, "ui-id-2")))
destino_primer_elemento = destino_ul_element.find_element(By.CSS_SELECTOR, "li:first-child")
destino_primer_elemento.click()

#fecha de ida
# Extraer el día, mes y año de la fecha ingresada
dia_ida, mes_ida, año_ida = fecha_ida_usuario.split('/')
dia_ida = int(dia_ida)
mes_ida = int(mes_ida)

# Abrir el selector de fecha
fecha_de_ida_boton = driver.find_element(By.ID, '_JourneySearchPortlet_WAR_Alsaportlet_INSTANCE_JourneySearch_21651890_departureDate_bt')
fecha_de_ida_boton.click()

# Esperar que se abra el calendario
calendario_ida_abierto = esperar.until(
    EC.visibility_of_element_located((By.ID, 'containerDatepickerOutward')))

# Ajustar el calendario al mes y año correctos
# Verificar si el mes del calendario actual es el que el usuario eligió
mes_calendario_ida = driver.find_element(By.CSS_SELECTOR, 'div.ui-datepicker-title > span.ui-datepicker-month').text
mes_calendario_ida = mes_a_numero(mes_calendario_ida)

# Si el mes y el año no coinciden con lo que el usuario ha elegido, navegar al mes y año correcto
while mes_calendario_ida != mes_ida:
    siguiente_mes_ida = driver.find_element(By.CSS_SELECTOR, 'a.ui-datepicker-next')
    siguiente_mes_ida.click()

    # Actualizar el mes y año del calendario después de navegar
    mes_calendario_ida = driver.find_element(By.CSS_SELECTOR, 'div.ui-datepicker-title > span.ui-datepicker-month').text
    mes_calendario_ida = mes_a_numero(mes_calendario_ida)

# Ahora seleccionar el día del mes elegido
fecha_ida_elemento = driver.find_element(By.XPATH, f"//a[text()='{dia_ida}']")
fecha_ida_elemento.click()

#fecha de vuelta  
# Extraer el día, mes y año de la fecha ingresada
dia_vuelta, mes_vuelta, año_vuelta = fecha_vuelta_usuario.split('/')
dia_vuelta = int(dia_vuelta)
mes_vuelta = int(mes_vuelta)

# Abrir el selector de fecha
fecha_de_vuelta_boton = driver.find_element(By.ID, '_JourneySearchPortlet_WAR_Alsaportlet_INSTANCE_JourneySearch_21651890__returnDate_bt')
fecha_de_vuelta_boton.click()

# Esperar que se abra el calendario
calendario_vuelta_abierto = esperar.until(
    EC.visibility_of_element_located((By.ID, 'containerDatepickerReturn')))

volver_boton = driver.find_element(By.CSS_SELECTOR, '#secondPartJourneySearchForm > div.form-group.second.icn-calendar > div > div:nth-child(4) > div > a') 
volver_boton.click()
fecha_de_vuelta_boton.click()

calendario_vuelta_abierto = esperar.until(
    EC.visibility_of_element_located((By.ID, 'containerDatepickerReturn')))

# Ajustar el calendario al mes y año correctos
# Verificar si el mes del calendario actual es el que el usuario eligió
mes_calendario_vuelta = driver.find_element(By.CSS_SELECTOR, 'div.ui-datepicker-title > span.ui-datepicker-month').text
mes_calendario_vuelta = mes_a_numero(mes_calendario_vuelta)

# Si el mes y el año no coinciden con lo que el usuario ha elegido, navegar al mes y año correcto
while mes_calendario_vuelta != mes_vuelta:
    siguiente_mes_vuelta = driver.find_element(By.CSS_SELECTOR, 'a.ui-datepicker-next')
    siguiente_mes_vuelta.click()

    # Actualizar el mes y año del calendario después de navegar
    mes_calendario_vuelta = driver.find_element(By.CSS_SELECTOR, 'div.ui-datepicker-title > span.ui-datepicker-month').text
    mes_calendario_vuelta = mes_a_numero(mes_calendario_vuelta)

# Ahora seleccionar el día del mes elegido
fecha_vuelta_elemento = driver.find_element(By.XPATH, f"//a[text()='{dia_vuelta}']")
fecha_vuelta_elemento.click()

mostrar_viajes = driver.find_element(By.CSS_SELECTOR, '#journeySearchFormButtonjs')
mostrar_viajes.click()

time.sleep(10)

# Esperar hasta que se carguen los elementos de los viajes
viajes = esperar.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.content-journey.body-row.row.slide-in-results.ng-scope.slide-in-results-key-frames')))

# Lista para almacenar los datos de los viajes
viaje_data = []

for viaje in viajes:
    try:
        # Extraer los datos de cada viaje
        origin = viaje.find_element(By.CSS_SELECTOR, 'p.destinos > span.resaltado.left.icn-arrow-right.ng-binding').text
        destination = viaje.find_element(By.CSS_SELECTOR, 'p.destinos > span.right.ng-binding').text
        departure_time = viaje.find_element(By.CSS_SELECTOR, 'p.return.ng-scope > span.hora.hora-salida.left.ng-binding').text
        arrival_time = viaje.find_element(By.CSS_SELECTOR, 'p.return.ng-scope > span.hora.hora-llegada.right.ng-binding').text
        duration = viaje.find_element(By.CSS_SELECTOR, 'p.return.ng-scope > span.time-travel > span.time-travel.time.ng-binding').text
        price = viaje.find_element(By.CSS_SELECTOR, "span.bottom.btn.btn-primary.ng-binding.ng-scope").text

        # Crear un diccionario para el viaje actual
        viaje_dict = {
            'origen': origin,
            'destino': destination,
            'hora_de_salida': departure_time,
            'hora_de_llegada': arrival_time,
            'duracion': duration,
            'precio': price
        }

        # Agregar el diccionario a la lista
        if all(viaje_dict.values()):
            viaje_data.append(viaje_dict)

    except:
        pass

numero_viajes_de_ida = len(viaje_data)
# Imprimir los datos recopilados
print(f'Numero de viajes de ida: {numero_viajes_de_ida}')

for viaje in viaje_data:
    print(viaje)

time.sleep(10)    

primer_billete = driver.find_element(By.CSS_SELECTOR, '#defaultFareOutward0 > span.precio-billete')
primer_billete.click()

time.sleep(10)

# Esperar hasta que se carguen los elementos de los viajes
viajes_vuelta = esperar.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.content-journey.body-row.row.slide-in-results.ng-scope.slide-in-results-key-frames')))

# Lista para almacenar los datos de los viajes
viaje_vuelta_data = []

for viaje in viajes_vuelta:
    try:
        # Extraer los datos de cada viaje
        origin = viaje.find_element(By.CSS_SELECTOR, 'p.destinos > span.resaltado.left.icn-arrow-right.ng-binding').text
        destination = viaje.find_element(By.CSS_SELECTOR, 'p.destinos > span.right.ng-binding').text
        departure_time = viaje.find_element(By.CSS_SELECTOR, 'p.return.ng-scope > span.hora.hora-salida.left.ng-binding').text
        arrival_time = viaje.find_element(By.CSS_SELECTOR, 'p.return.ng-scope > span.hora.hora-llegada.right.ng-binding').text
        duration = viaje.find_element(By.CSS_SELECTOR, 'p.return.ng-scope > span.time-travel > span.time-travel.time.ng-binding').text
        price = viaje.find_element(By.CSS_SELECTOR, "span.bottom.btn.btn-primary.ng-binding.ng-scope").text

        # Crear un diccionario para el viaje actual
        viaje_vuelta_dict = {
            'origen': origin,
            'destino': destination,
            'hora_de_salida': departure_time,
            'hora_de_llegada': arrival_time,
            'duracion': duration,
            'precio': price
        }

        # Agregar el diccionario a la lista
        if all(viaje_vuelta_dict.values()):
            viaje_vuelta_data.append(viaje_vuelta_dict)

    except:
        pass

numero_viajes_de_vuelta = len(viaje_vuelta_data)
# Imprimir los datos recopilados
print(f'Numero de viajes de vuelta: {numero_viajes_de_vuelta}')

for viaje in viaje_vuelta_data:
    print(viaje)



ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF77765FB05+28789]
	(No symbol) [0x00007FF7775C86E0]
	(No symbol) [0x00007FF777465759]
	(No symbol) [0x00007FF7774BA7C4]
	(No symbol) [0x00007FF7774ACD60]
	(No symbol) [0x00007FF7774DF43A]
	(No symbol) [0x00007FF7774AC5B6]
	(No symbol) [0x00007FF7774DF650]
	(No symbol) [0x00007FF7774FF654]
	(No symbol) [0x00007FF7774DF1E3]
	(No symbol) [0x00007FF7774AA938]
	(No symbol) [0x00007FF7774ABAA1]
	GetHandleVerifier [0x00007FF77799933D+3410093]
	GetHandleVerifier [0x00007FF7779AE7DD+3497293]
	GetHandleVerifier [0x00007FF7779A2A73+3448803]
	GetHandleVerifier [0x00007FF777727BBB+848171]
	(No symbol) [0x00007FF7775D3C3F]
	(No symbol) [0x00007FF7775CF6E4]
	(No symbol) [0x00007FF7775CF87D]
	(No symbol) [0x00007FF7775BED49]
	BaseThreadInitThunk [0x00007FFC00FB7374+20]
	RtlUserThreadStart [0x00007FFC01D7CC91+33]


 Hola, somos Manuel, Mateo y Vicente. Ante la reciente aparición del verano Joven, pretendemos facilitar su uso en la selección de viajes en tren y autobus. Por ello queremos mantener informados a nuestros usuarios sobre viajes de tres y autobus de toda españa.


¿Cual es tu medio de transporte? (tren/autobus) autobus
fecha de salida (DD/MM/AAAA) 02/02/2025
fecha de vuelta (DD/MM/AAAA) 03/02/2025ç
Ciudad de origen Madrid
Ciudad de destino Barcelona


Numero de viajes de ida: 38
{'origen': 'Madrid Interc. Av América', 'destino': 'Barcelona Estación Sants', 'hora_de_salida': '01:00', 'hora_de_llegada': '08:50', 'duracion': '07:50 h', 'precio': '19,00 €'}
{'origen': 'Madrid Interc. Av América', 'destino': 'Barcelona Estación Nord', 'hora_de_salida': '01:00', 'hora_de_llegada': '09:05', 'duracion': '08:05 h', 'precio': '19,00 €'}
{'origen': 'Aeropuerto Madrid - Barajas T4', 'destino': 'Barcelona Estación Sants', 'hora_de_salida': '01:15', 'hora_de_llegada': '08:50', 'duracion': '07:35 h', 'precio': '38,42 €'}
{'origen': 'Aeropuerto Madrid - Barajas T4', 'destino': 'Barcelona Estación Nord', 'hora_de_salida': '01:15', 'hora_de_llegada': '09:05', 'duracion': '07:50 h', 'precio': '38,42 €'}
{'origen': 'Madrid Interc. Av América', 'destino': 'Barcelona Estación Sants', 'hora_de_salida': '08:00', 'hora_de_llegada': '16:05', 'duracion': '08:05 h', 'precio': '9,00 €'}
{'origen': 'Madrid Interc. Av América', 'destino': 'Barcelona Estación Nord

In [5]:
def autobus():   
    